In [1]:
import os
from os.path import join
import nibabel as nib
from skimage.measure import label, regionprops
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [2]:
def connected_component_analysis(seg, thr=0.5):
    #print(type(seg), seg.shape, seg.min(), seg.max())
    mask = seg.copy()
    mask[mask > thr] = 255
    mask[mask <= thr] = 0

    lbl = label(mask)
    blobs = regionprops(lbl)
    #raw_bbox_list = [b.bbox for b in blobs]
    #axis_major_length_list = [b.axis_major_length for b in blobs]

    return blobs

def cystregionprops(r):
    area = r.area
    major = round(r.axis_major_length, 3)
    
    try:
        minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
    except ValueError:
        minor = np.nan

    return (area, major, minor)


In [3]:
# Folder path
#seg_folder_path = r'C:\Users\pky0507\Desktop\ilkin\pancreas_ipmn\temporary_folder_cyst_segmentations\allegheny\cyst_segmentation'
#mri_folder_path = r'C:\Users\pky0507\Desktop\ilkin\pancreas_ipmn\allegheny\reoriented'

seg_folder_path = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/northwestern/cyst_segmentation'
mri_folder_path = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/northwestern/reoriented'

# List to store file paths
seg_file_paths = []
mri_file_paths = []
ids = []

# Iterate over files in the folder
for root, dirs, files in os.walk(seg_folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        if file.startswith(".") : continue
        seg_file_paths.append(file_path)

# Print the file paths
for path in seg_file_paths:
    id = path.split("/")[-1].split(".")[:-2][-2:]
    if id==[]: continue
    id = int(id[-1][-2:])
    print(id)
    mri_file_paths.append(os.path.join(mri_folder_path, f"Patient-{id:04d}.nii.gz"))


2
8
15
12
19
9
18
11
7
3
6
20
16
17
1
14
13
4
10
5


In [4]:
for i in range(len(seg_file_paths)):
    print(f"pair {i+1}")
    print(seg_file_paths[i-1], "\n", mri_file_paths[i-1])

#print(len(seg_file_paths), len(mri_file_paths))

pair 1
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/northwestern/cyst_segmentation/Patient_0005.nii.gz 
 /data/ayc9699/dataset/pancreas_ipmn_ilkin/northwestern/reoriented/Patient-0005.nii.gz
pair 2
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/northwestern/cyst_segmentation/Patient_0002.nii.gz 
 /data/ayc9699/dataset/pancreas_ipmn_ilkin/northwestern/reoriented/Patient-0002.nii.gz
pair 3
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/northwestern/cyst_segmentation/Patient_0008.nii.gz 
 /data/ayc9699/dataset/pancreas_ipmn_ilkin/northwestern/reoriented/Patient-0008.nii.gz
pair 4
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/northwestern/cyst_segmentation/Patient_0015.nii.gz 
 /data/ayc9699/dataset/pancreas_ipmn_ilkin/northwestern/reoriented/Patient-0015.nii.gz
pair 5
/data/ayc9699/dataset/pancreas_ipmn_ilkin/temporary_folder_cyst_segmentations/northwestern/cyst_s

In [6]:

def create_df():
    column_names=['Center', 'Patient', 'Pancreas_volume_ml', 'Cyst_volume_ml', 'Panc_Cyst_volume_ml', 'Diagonal_mm', 'Panc_vol_to_Diagonal', 'Num_of_Cysts']
    # Add columns using a for loop

    # Define the number of cysts
    num_cysts = 3
    for i in range(num_cysts):
        area_column_name = f'Cyst_{i+1}_vol_ml'
        major_column_name = f'Cyst_{i+1}_major_mm'
        minor_column_name = f'Cyst_{i+1}_minor_mm'
        column_names.append(area_column_name)
        column_names.append(major_column_name)
        column_names.append(minor_column_name)

    # Create an empty DataFrame
    df = pd.DataFrame(columns = column_names)

    df.head()
    return df

In [7]:

def largest3rs(r_list, voxel_sizes):

    voxel_sizes = [voxel_sizes[2], voxel_sizes[1], voxel_sizes[0]]
    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)
    top_3 = []
    
    for r in sorted_r[:3]:
      
        cov_matrix = r.inertia_tensor

        # Perform eigendecomposition to obtain eigenvectors and eigenvalues
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

        # Determine the dimension along which the major and minor axes lie
        major_axis_dimension = np.argmax(eigenvalues)
        minor_axis_dimension = np.argmin(eigenvalues)

        # Multiply the measurements by the corresponding voxel sizes
        axis_major_length_mm = r.major_axis_length * voxel_sizes[major_axis_dimension]
        try:
            axis_minor_length_mm = r.minor_axis_length * voxel_sizes[minor_axis_dimension]
        except ValueError:
            axis_minor_length_mm = np.nan
        
        # Use the calculated measurements and axis dimensions as needed
        #print(f"Axis Major Length (mm): {axis_major_length_mm}, Dimension: {major_axis_dimension}")
        #print(f"Axis Minor Length (mm): {axis_minor_length_mm}, Dimension: {minor_axis_dimension}")

        vol_ml = r.area * voxel_sizes[0] * voxel_sizes[1] * voxel_sizes[2]/1000

        #major = round(r.axis_major_length, 3)
        
        axis_major_length_mm = round(axis_major_length_mm, 3)
        axis_minor_length_mm = round(axis_minor_length_mm, 3)
        
        '''
        try:
            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
        except ValueError:
            minor = np.nan
        '''        
        
        top_3.append((vol_ml, axis_major_length_mm, axis_minor_length_mm))
        #top_3.append((vol_ml, major, minor))

    
    return top_3


In [24]:
'''
def largest3rs(r_list):
    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)
    top_3 = []
    
    for r in sorted_r[:3]:
        area = r.area
        major = round(r.axis_major_length, 3)
        
        try:
            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)
        except ValueError:
            minor = np.nan
        
        top_3.append((area, major, minor))
    
    return top_3
    '''


'\ndef largest3rs(r_list):\n    sorted_r = sorted(r_list, key=lambda r: r.axis_major_length, reverse=True)\n    top_3 = []\n    \n    for r in sorted_r[:3]:\n        area = r.area\n        major = round(r.axis_major_length, 3)\n        \n        try:\n            minor = round(math.sqrt(10 * (-r.inertia_tensor_eigvals[0] + r.inertia_tensor_eigvals[1] + r.inertia_tensor_eigvals[2])), 3)\n        except ValueError:\n            minor = np.nan\n        \n        top_3.append((area, major, minor))\n    \n    return top_3\n    '

In [8]:
from skimage.measure import label, regionprops
import numpy as np
def compute_diagonal_length_3D(image, voxel_size_x, voxel_size_y, voxel_size_z):
    # Ensure that image is a binary image
    #print(image.shape)
    assert np.array_equal(image, image.astype(bool)), "Input should be a binary image"

    # Label the image
    labeled_image = label(image)
    #print(np.unique(labeled_image))

    # Compute the properties of the labeled regions
    regions = regionprops(labeled_image)

    # Initialize list to store diagonal lengths
    diagonals = []
    #print("voxel",voxel_size_x, voxel_size_y, voxel_size_z)
    for region in regions:
        # Get the bounding box coordinates
        minp, minr, minc, maxp, maxr, maxc = region.bbox
        # Compute the width, height, and depth of the box
        width = (maxp - minp) * voxel_size_x
        height = (maxr - minr) * voxel_size_y
        depth = (maxc - minc) * voxel_size_z
        #print("debug--------", "depth:", (maxc - minc), "y ekseni:", (maxr - minr),  "x ekseni:", (maxp - minp))
        #print(region.bbox, width, height, depth)

        # Use Pythagorean theorem to compute the diagonal length
        diagonal = np.sqrt(width**2 + height**2 + depth**2)
        diagonals.append(diagonal)

    return diagonals

In [9]:
def get_volume_ml(label, voxel_vol_mm3):
    volume_mm3 = np.sum(label != 0) * voxel_vol_mm3
    volume_ml = volume_mm3 / 1000
    return volume_ml

In [10]:
#fig, axs = plt.subplots(1, 3, figsize=(15, 5))

df = create_df()

for eachpatientind in range(len(seg_file_paths)):
    
    pt_name = seg_file_paths[eachpatientind].split("/")[-1].split(".")[0][-2:]
    print(pt_name)
    #if seg_file_paths[eachpatientind].split("\\")[-1].split("_")[-1] == 'pass.nii': continue
    #if pt_name == '0089': continue
    # Read niftii image and segmentation
    #print(pt_name)
    #img_nib = nib.load(mri_file_paths[eachpatientind])
    seg_nib = nib.load(seg_file_paths[eachpatientind])
    
    #print(pt_name, img_nib.header.get_zooms(), seg_nib.header.get_zooms())
    #print(pt_name, img_nib.shape, seg_nib.shape)

    #if pt_name in ['0140', '0159', '0172']:
        #print(seg_file_paths[eachpatientind].split("\\")[-1])
        #print(pt_name, img_nib.header.get_zooms(), seg_nib.header.get_zooms())
    #else:
    #assert img_nib.header.get_zooms() == seg_nib.header.get_zooms()
    #assert img_nib.shape == seg_nib.shape

    voxel_sizes = seg_nib.header.get_zooms()
    print(voxel_sizes)

    #img = img_nib.get_fdata()
    seg = seg_nib.get_fdata()

    print(np.unique(seg))

    #print('img', img.shape)
    #print('seg', seg.shape)

    cysts = (seg == 9999)
    pancreas = (seg != 0)
    cysts_b = cysts.astype(int)
    pancreas_b = pancreas.astype(int)
    pancreasandcyst = np.concatenate((cysts_b, pancreas_b))

    # Find Bboxes
    blobs = connected_component_analysis(cysts, thr=0.5)
    num_of_cysts = len(blobs)

    # Calculate the voxel volume (assuming isotropic voxel size)
    voxel_vol_mm3 = voxel_sizes[0] *voxel_sizes[1] * voxel_sizes[2]  # Specify the voxel volume in your desired units

    # Calculate the volume of the segmentation mask
    cysts_volume_ml = get_volume_ml(cysts, voxel_vol_mm3)
    pancreas_volume_ml = get_volume_ml(pancreas, voxel_vol_mm3)

    diagonal_lenght = max(compute_diagonal_length_3D(pancreasandcyst, voxel_sizes[0], voxel_sizes[1], voxel_sizes[2]))
    pancreas_volume_to_diagonal = pancreas_volume_ml / diagonal_lenght

    column_data = ['Northwestern', pt_name, pancreas_volume_ml, cysts_volume_ml, pancreas_volume_ml + cysts_volume_ml, diagonal_lenght, pancreas_volume_to_diagonal, num_of_cysts]

    result = largest3rs(blobs, voxel_sizes)
    # Print the result
    for i, props in enumerate(result, start=1):
        #print(f"Top {i}: Area={props[0]}, Major={props[1]}, Minor={props[2]}")
        area, major, minor = props[0], props[1], props[2]
        column_data.append(area)
        column_data.append(major)
        column_data.append(minor)

    column_data.extend([math.nan] * (17 - len(column_data)))
    #print(column_data)

    df.loc[len(df)] = column_data

df.head()

#plt.show()

02
(1.484375, 1.484375, 7.7999897)
[0. 5.]


08
(1.4257812, 1.4257812, 6.0)
[0. 1.]
15
(1.03125, 1.03125, 5.0)
[0. 1.]
12
(1.40625, 1.40625, 5.0)
[0. 5.]
19
(1.1875, 1.1875, 5.0)
[0. 1.]
09
(1.15625, 1.15625, 5.0)
[0. 1.]
18
(1.25, 1.25, 6.600006)
[0. 1.]
11
(1.3125, 1.3125, 5.0)
[0. 1.]
07
(1.25, 1.25, 6.600006)
[0. 5.]
03
(1.34375, 1.34375, 6.599997)
[0. 1.]
06
(1.125, 1.125, 6.0)
[0. 1.]
20
(1.1875, 1.1875, 6.0)
[0. 1.]
16
(1.1875, 1.1875, 7.199997)
[0. 1.]
17
(1.15625, 1.15625, 5.0)
[0. 5.]
01
(1.09375, 1.09375, 5.5)
[0. 1.]
14
(1.125, 1.125, 6.0)
[0. 5.]
13
(1.125, 1.125, 6.0)
[0. 1.]
04
(1.40625, 1.40625, 5.0)
[0. 1.]
10
(1.09375, 1.09375, 4.999992)
[0. 1.]
05
(1.1875, 1.1875, 7.199997)
[0. 1.]


,Center,Patient,Pancreas_volume_ml,Cyst_volume_ml,Panc_Cyst_volume_ml,Diagonal_mm,Panc_vol_to_Diagonal,Num_of_Cysts,Cyst_1_vol_ml,Cyst_1_major_mm,Cyst_1_minor_mm,Cyst_2_vol_ml,Cyst_2_major_mm,Cyst_2_minor_mm,Cyst_3_vol_ml,Cyst_3_major_mm,Cyst_3_minor_mm
0,Northwestern,02,84.367333,0.0,84.367333,164.096665,0.514132,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Northwestern,08,35.396022,0.0,35.396022,118.156994,0.299568,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Northwestern,15,23.678306,0.0,23.678306,124.078577,0.190833,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Northwestern,12,64.972046,0.0,64.972046,173.288576,0.374936,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Northwestern,19,49.870176,0.0,49.870176,191.861359,0.259928,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Save the DataFrame to a CSV file
df.to_csv('server_cyst_segmentation_northwestern.csv', index=False)